###**Cel ćwiczenia:**
Celem przeprowadzonego ćwiczenia jest rozwiązanie problemu liniowego. Naszym zadaniem jest znalezienie optymalnego rozwiązania jeśli chodzi o ilości produkcji okien aluminiowych lub drewnianych, aby osiągnąć maksymalny zysk. Zysk wynoszący z produkcji jednej partii okien drewnianych wynosi 5000zł, natomiast z produkcji jednej partii okien aluminiowych 3000zł. Producent posiada trzy fabryki. Każda z tych fabryk posiada ograniczenia w możliwości produkcji danego typu okien. Następujące ograniczenia przedstawia tabela:

|           | okna drewniane(czas potrzebny na produkcję 1 partii) | okna aluminiowe(czas potrzebny na produkcję 1 partii) | dostępne godziny |
|-----------|------------------------------------------------------|-------------------------------------------------------|------------------|
| fabryka 1: |                    1                          |                           x                           |         4        |
| fabryka 2: |                           x                          |                           2                           |        12        |
| fabryka 3: |                           3                          |                           2                           |        18        |

Posiadając takie dane musimy rozwiązać zadanie problemu liniowego jak rozłożyć produkcję poszczególnych okien, tak aby zysk był jak największy. Maksymalizujemy zysk z produkcji.



In [1]:
!pip install pulp

#!sudo apt-get install coinor-cbc glpk-utils coinor-clp
!conda install -c conda-forge pulp

     |████████████████████████████████| 40.6MB 114kB/s 
/bin/bash: conda: command not found


In [2]:
import pulp

print(pulp.__version__)

2.3.1


In [3]:
from pulp import *

In [4]:
prob = LpProblem("Okna",LpMaximize)
x1=LpVariable("Okna_aluminiowe",0) #stworzenie zmiennych decyzyjnych 
x2=LpVariable("Okna_drewniane",0)
prob += 3000*x1 + 5000*x2, "Zysk" #obliczenie zysku 
prob += x1 <= 4, "Fabryka_1" #ograniczenia fabryki 1
prob += 2*x2 <= 12, "Fabryka_2" ##ograniczenia fabryki 2
prob += 3*x1 + 2*x2 <= 18, "Fabrryka_3" #ograniczenia fabryki 3


prob.writeLP("Okna.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Zysk = ", value(prob.objective))

Status: Optimal
Okna_aluminiowe = 2.0
Okna_drewniane = 6.0
Zysk =  36000.0


Obliczenie zysku jest realizowane przez wymnożenie ilości partii wyprodukowanych przez zysk jaki generuje dana partia. Zastosowane ograniczenia wynikają z danych zawartych w tabeli. Optymalnym rozwiązaniem naszego problemu jest wytworzenie przez danego producenta 2 partii okien aluminiowych oraz 6 partii okien drewnianych. Wyliczony maksymalny zysk wyniesie 36000zł. W dalszej częsci ćwiczenia zostały dodane suwaki, dzięki którymi mamy możliwość zmiany ograniczenia w godzinach pracy poszczególnych fabryk.

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}
fabryka_1_slider = FloatSlider(min=0,max = 50, value= 4, description="fabryka_1_slider", style= style ) #zdefiniowanie ograniczeń godzinowych pracy poszczególnych fabryk jako wielkości slider
fabryka_2_slider = FloatSlider(min=0,max = 50,value= 12, description="fabryka_2_slider", style= style )
fabryka_3_slider = FloatSlider(min=0,max = 50, value= 18, description="fabryka_3_slider", style= style )
def hurt_produkcja(fabryka_1 = 4, #zdefiniowanie wartości ograniczeń początkowe
    fabryka_2 = 12,
    fabryka_3= 18,):
  
  prob = LpProblem("Okna",LpMaximize)
  x1=LpVariable("Okna_aluminiowe",0, None, LpInteger) #zmienne decyzyjne
  x2=LpVariable("Okna_drewniane",0, None, LpInteger)
  prob += 3000*x1 + 5000*x2, "Zysk" #obliczanie zysku 
  prob += x1 <= fabryka_1, "Fabryka_1" # ograniczenia poszczególnych fabryk
  prob += 2*x2 <= fabryka_2, "Fabryka_2"
  prob += 3*x1 + 2*x2 <= fabryka_3, "Fabrryka_3"

  prob.writeLP("Okna.lp")
  prob.solve()
  print("Status:", LpStatus[prob.status])
  for v in prob.variables():
    print(v.name, "=", v.varValue)
  print("Zysk = ", value(prob.objective))
  shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
  print("*"*10+"Shadow Prices"+"*"*10)
  print(pd.DataFrame(shadows))
interact(hurt_produkcja,fabryka_1 = fabryka_1_slider, # przypisanie do zmiennych określających ograniczenia wartości odczytanych z suwaków
    fabryka_2 = fabryka_2_slider,
    fabryka_3 = fabryka_3_slider,)

interactive(children=(FloatSlider(value=4.0, description='fabryka_1_slider', max=50.0, style=SliderStyle(descr…

<function __main__.hurt_produkcja>

Zmieniając wartości poszczególnych suwaków zmieniamy ilość godzin dostępnych w poszczególnych fabrykach. Dzięki temu możliwa jest obserwacja zmian otrzymanego maksymalnego zysku, a także ilości produkowanych partii okien w zależności od wydajności danej firmy. Tabela shadow price informuje nas czy wartości ograniczeń są optymalne, czy możliwy jest lepszy dobór danych wartości. 